In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [3]:
data = pd.read_csv(".train.csv")

FileNotFoundError: [Errno 2] No such file or directory: '.train.csv'

In [ ]:
data.head()

In [ ]:
data = data[["POSTED_BY", "BHK_NO.", "BHK_OR_RK", "SQUARE_FT", "RESALE", "LONGITUDE", "LATITUDE", "TARGET(PRICE_IN_LACS)"]]
data.head()

In [ ]:
data.rename(columns={
    "BHK_NO.": "bkh_no",
    "TARGET(PRICE_IN_LACS)": "price"
}, inplace=True)

data.rename(columns={k:v for k, v in zip(data.columns, [i.lower() for i in data.columns])}, inplace=True)

In [ ]:
data.head()

In [ ]:
data["bhk_or_rk"].value_counts()

In [ ]:
data = data[data["bhk_or_rk"] == "BHK"]
data.head()

In [ ]:
data["bhk_or_rk"].value_counts()

In [ ]:
data.drop(columns=["bhk_or_rk"], inplace=True, axis=1)

In [ ]:
data.head()

In [ ]:
data.isna().sum()

In [ ]:
data.duplicated().sum()

In [ ]:
data.drop_duplicates(inplace=True, keep="first")

In [ ]:
data['price'] = data['price'].apply(lambda x: x*100000).astype("int32")

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data=data[(data['price'] > 90000) & (data['price'] < 90000000)]
data.head()

In [ ]:
data.describe()

In [ ]:
data = data[data["bkh_no"] <= 10]

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data = data[data["square_ft"] > 500]

In [ ]:
data.describe()

In [ ]:
data['price_per_sqrft'] = data['price']/data['square_ft'].astype('float32')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
## Removing outliers using price_per_sqrt column

mn = np.mean(data['price_per_sqrft'])
stnd = np.std(data['price_per_sqrft'])

lower_bound = mn - 3*stnd
upper_bound = mn + 3*stnd

lower_bound, upper_bound

In [ ]:
data = data[(data['price_per_sqrft'] < 35850.93592917894) & (data['price_per_sqrft'] > 2000)]

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
data["posted_by"].value_counts()

In [ ]:
data = pd.concat([data, pd.get_dummies(data['posted_by'], drop_first=True)], axis=1)

In [ ]:
data.head()

In [ ]:
data.drop(columns=["posted_by", "price_per_sqrft"], inplace=True, axis=1)
data.head()

In [ ]:
plt.figure(figsize=(6,4))
sns.distplot(data['price'])
plt.show()

In [ ]:
## As we can see that Price column is left skewed. Because majority of data is in left side. As we will use np.log to make it normally distributed

plt.figure(figsize=(6,4))
sns.distplot(np.log1p(data['price']))
plt.show()

In [ ]:
data['price'] = np.log1p(data['price'])

In [ ]:
data.to_csv("./final_cleaned_train.csv", index=False)

In [ ]:
data.head()

In [ ]:
ss = StandardScaler()
columns = ["square_ft", "longitude", "latitude"]

data[columns] = ss.fit_transform(data[columns])
data.head()

In [ ]:
X = data.drop(columns=['price'], axis=1)
y = data['price']

X.dropna(inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
X_train.dropna(axis=0, inplace=True)

In [ ]:
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
rfr_train_pred = rfr.predict(X_train)
train_acc_rfr = r2_score(y_train, rfr_train_pred)
rfr_test_pred = rfr.predict(X_test)
test_acc_rfr = r2_score(y_test, rfr_test_pred)

print("Train accuracy of RFR :-> ", train_acc_rfr)
print("Test accuracy of RFR :-> ", test_acc_rfr)

In [ ]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
xgb_train_pred = xgb.predict(X_train)
train_acc_xgb = r2_score(y_train, xgb_train_pred)

xgb_test_pred = xgb.predict(X_test)
test_acc_xgb = r2_score(y_test, xgb_test_pred)

print("Train acc:-> ", train_acc_xgb)
print("Test acc:-> ", test_acc_xgb)